<a href="https://colab.research.google.com/github/Pranav64bit/RandomStuff/blob/main/cbcb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests geopy pandas numpy scikit-learn -q

In [2]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
from geopy.geocoders import Nominatim
import warnings
warnings.filterwarnings('ignore')

In [3]:
class WeatherAPIClient:
    """
    Client to fetch real-time weather data from Open-Meteo API
    """
    def __init__(self):
        self.base_url = "https://api.open-meteo.com/v1/forecast"
        self.geocoder = Nominatim(user_agent="cloudburst_detector_v1")

    def get_coordinates(self, place_name):
        """
        Convert place name to latitude and longitude
        """
        try:
            location = self.geocoder.geocode(place_name + ", India")
            if location:
                return location.latitude, location.longitude, location.address
            else:
                return None, None, None
        except Exception as e:
            print(f"Error getting coordinates: {e}")
            return None, None, None

    def fetch_weather_data(self, latitude, longitude):
        """
        Fetch comprehensive weather data from Open-Meteo API
        """
        try:
            params = {
                'latitude': latitude,
                'longitude': longitude,
                'current': 'temperature_2m,relative_humidity_2m,precipitation,rain,showers,snowfall,weather_code,cloud_cover,pressure_msl,surface_pressure,wind_speed_10m,wind_direction_10m,wind_gusts_10m',
                'hourly': 'temperature_2m,relative_humidity_2m,dew_point_2m,precipitation_probability,precipitation,rain,showers,pressure_msl,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,wind_speed_10m',
                'timezone': 'Asia/Kolkata',
                'forecast_days': 1
            }

            response = requests.get(self.base_url, params=params, timeout=10)
            response.raise_for_status()
            return response.json()

        except requests.exceptions.RequestException as e:
            print(f"Error fetching weather data: {e}")
            return None

In [4]:
class CloudBurstFloodPredictor:
    """
    Advanced predictor for cloud burst and flash flood risks
    """
    def __init__(self):
        self.risk_factors = {
            'precipitation_rate': 0.25,
            'humidity': 0.20,
            'pressure_drop': 0.20,
            'cloud_cover': 0.15,
            'wind_speed': 0.10,
            'temperature': 0.10
        }

    def calculate_precipitation_risk(self, precipitation, rain, showers):
        """
        Calculate risk based on precipitation intensity
        Cloud burst: >100mm in 1-2 hours
        """
        total_precip = precipitation + rain + showers

        if total_precip >= 100:
            return 100
        elif total_precip >= 50:
            return 85
        elif total_precip >= 30:
            return 65
        elif total_precip >= 15:
            return 45
        elif total_precip >= 5:
            return 25
        else:
            return 10

    def calculate_humidity_risk(self, humidity):
        """
        High humidity increases cloud burst risk
        """
        if humidity >= 90:
            return 90
        elif humidity >= 80:
            return 70
        elif humidity >= 70:
            return 50
        elif humidity >= 60:
            return 30
        else:
            return 15

    def calculate_pressure_risk(self, current_pressure, surface_pressure):
        """
        Rapid pressure drops indicate severe weather
        """
        pressure_diff = abs(current_pressure - surface_pressure)

        if pressure_diff >= 15:
            return 85
        elif pressure_diff >= 10:
            return 65
        elif pressure_diff >= 5:
            return 40
        else:
            return 20

    def calculate_cloud_cover_risk(self, cloud_cover):
        """
        Dense cloud cover increases risk
        """
        if cloud_cover >= 90:
            return 80
        elif cloud_cover >= 75:
            return 60
        elif cloud_cover >= 50:
            return 40
        else:
            return 20

    def calculate_wind_risk(self, wind_speed, wind_gusts):
        """
        High wind speeds can indicate severe weather systems
        """
        max_wind = max(wind_speed, wind_gusts)

        if max_wind >= 50:
            return 75
        elif max_wind >= 35:
            return 55
        elif max_wind >= 20:
            return 35
        else:
            return 15

    def calculate_temperature_risk(self, temperature):
        """
        Extreme temperatures can contribute to severe weather
        """
        if temperature >= 35 or temperature <= 5:
            return 60
        elif temperature >= 30 or temperature <= 10:
            return 40
        else:
            return 20

    def calculate_hourly_trends(self, hourly_data):
        """
        Analyze hourly trends for next 6 hours
        """
        if not hourly_data:
            return 0

        precip_trend = np.mean(hourly_data.get('precipitation', [0]*6)[:6])
        humidity_trend = np.mean(hourly_data.get('relative_humidity_2m', [0]*6)[:6])

        trend_risk = 0
        if precip_trend > 10:
            trend_risk += 30
        if humidity_trend > 85:
            trend_risk += 20

        return min(trend_risk, 50)

    def predict_cloudburst_risk(self, weather_data):
        """
        Calculate overall cloud burst risk percentage
        """
        current = weather_data.get('current', {})
        hourly = weather_data.get('hourly', {})

        # Extract parameters
        precipitation = current.get('precipitation', 0)
        rain = current.get('rain', 0)
        showers = current.get('showers', 0)
        humidity = current.get('relative_humidity_2m', 0)
        pressure_msl = current.get('pressure_msl', 1013)
        surface_pressure = current.get('surface_pressure', 1013)
        cloud_cover = current.get('cloud_cover', 0)
        wind_speed = current.get('wind_speed_10m', 0)
        wind_gusts = current.get('wind_gusts_10m', 0)
        temperature = current.get('temperature_2m', 20)

        # Calculate individual risk scores
        precip_risk = self.calculate_precipitation_risk(precipitation, rain, showers)
        humidity_risk = self.calculate_humidity_risk(humidity)
        pressure_risk = self.calculate_pressure_risk(pressure_msl, surface_pressure)
        cloud_risk = self.calculate_cloud_cover_risk(cloud_cover)
        wind_risk = self.calculate_wind_risk(wind_speed, wind_gusts)
        temp_risk = self.calculate_temperature_risk(temperature)
        trend_risk = self.calculate_hourly_trends(hourly)

        # Weighted cloud burst risk calculation
        cloudburst_risk = (
            precip_risk * self.risk_factors['precipitation_rate'] +
            humidity_risk * self.risk_factors['humidity'] +
            pressure_risk * self.risk_factors['pressure_drop'] +
            cloud_risk * self.risk_factors['cloud_cover'] +
            wind_risk * self.risk_factors['wind_speed'] +
            temp_risk * self.risk_factors['temperature']
        )

        # Add trend bonus
        cloudburst_risk = min(cloudburst_risk + (trend_risk * 0.2), 100)

        return round(cloudburst_risk, 2)

    def predict_flood_risk(self, weather_data, cloudburst_risk):
        """
        Calculate flash flood risk based on weather conditions
        """
        current = weather_data.get('current', {})

        # Base flood risk from cloudburst risk
        flood_risk = cloudburst_risk * 0.7

        # Additional factors for flood risk
        precipitation = current.get('precipitation', 0) + current.get('rain', 0)

        # Heavy sustained precipitation increases flood risk
        if precipitation >= 50:
            flood_risk += 20
        elif precipitation >= 30:
            flood_risk += 15
        elif precipitation >= 15:
            flood_risk += 10

        # Terrain factor for Himachal and Uttarakhand (hilly terrain)
        terrain_multiplier = 1.15
        flood_risk *= terrain_multiplier

        return min(round(flood_risk, 2), 100)

    def get_risk_category(self, risk_percentage):
        """
        Categorize risk level
        """
        if risk_percentage >= 80:
            return "🔴 EXTREME - Immediate Action Required"
        elif risk_percentage >= 60:
            return "🟠 HIGH - Stay Alert"
        elif risk_percentage >= 40:
            return "🟡 MODERATE - Monitor Conditions"
        elif risk_percentage >= 20:
            return "🟢 LOW - Normal Conditions"
        else:
            return "⚪ MINIMAL - Safe Conditions"

In [5]:
class CloudBurstDetectionSystem:
    """
    Complete cloud burst detection system
    """
    def __init__(self):
        self.weather_client = WeatherAPIClient()
        self.predictor = CloudBurstFloodPredictor()

    def format_weather_report(self, weather_data, location_name):
        """
        Format weather data into readable report
        """
        current = weather_data.get('current', {})

        report = f"\n{'='*60}\n"
        report += f"📍 LOCATION: {location_name}\n"
        report += f"🕐 TIME: {current.get('time', 'N/A')}\n"
        report += f"{'='*60}\n\n"

        report += "🌡️ CURRENT WEATHER CONDITIONS:\n"
        report += f"   Temperature: {current.get('temperature_2m', 'N/A')}°C\n"
        report += f"   Humidity: {current.get('relative_humidity_2m', 'N/A')}%\n"
        report += f"   Precipitation: {current.get('precipitation', 0)}mm\n"
        report += f"   Rain: {current.get('rain', 0)}mm\n"
        report += f"   Cloud Cover: {current.get('cloud_cover', 'N/A')}%\n"
        report += f"   Wind Speed: {current.get('wind_speed_10m', 'N/A')} km/h\n"
        report += f"   Wind Gusts: {current.get('wind_gusts_10m', 'N/A')} km/h\n"
        report += f"   Pressure (MSL): {current.get('pressure_msl', 'N/A')} hPa\n"
        report += f"   Surface Pressure: {current.get('surface_pressure', 'N/A')} hPa\n"

        return report

    def analyze_location(self, place_name):
        """
        Complete analysis for a given location
        """
        print(f"\n🔍 Analyzing weather conditions for: {place_name}")
        print("⏳ Fetching real-time data...\n")

        # Get coordinates
        lat, lon, full_address = self.weather_client.get_coordinates(place_name)

        if lat is None or lon is None:
            print(f"❌ Error: Could not find location '{place_name}'")
            print("Please try with more specific location name (e.g., 'Shimla, Himachal Pradesh')")
            return None

        print(f"✅ Location found: {full_address}")
        print(f"   Coordinates: {lat:.4f}°N, {lon:.4f}°E\n")

        # Fetch weather data
        weather_data = self.weather_client.fetch_weather_data(lat, lon)

        if weather_data is None:
            print("❌ Error: Could not fetch weather data")
            return None

        # Generate weather report
        weather_report = self.format_weather_report(weather_data, full_address)
        print(weather_report)

        # Calculate risks
        print("\n⚠️ RISK ASSESSMENT:\n")

        cloudburst_risk = self.predictor.predict_cloudburst_risk(weather_data)
        flood_risk = self.predictor.predict_flood_risk(weather_data, cloudburst_risk)

        cloudburst_category = self.predictor.get_risk_category(cloudburst_risk)
        flood_category = self.predictor.get_risk_category(flood_risk)

        print(f"☁️ CLOUD BURST RISK: {cloudburst_risk}%")
        print(f"   Status: {cloudburst_category}\n")

        print(f"🌊 FLASH FLOOD RISK: {flood_risk}%")
        print(f"   Status: {flood_category}\n")

        # Risk interpretation
        print(f"{'='*60}")
        print("📊 RISK INTERPRETATION:")
        print(f"{'='*60}\n")

        if cloudburst_risk >= 60 or flood_risk >= 60:
            print("⚠️ WARNING: High risk conditions detected!")
            print("   Recommended actions:")
            print("   • Avoid travel to low-lying areas")
            print("   • Stay updated with weather alerts")
            print("   • Keep emergency supplies ready")
            print("   • Follow local authority instructions\n")
        elif cloudburst_risk >= 40 or flood_risk >= 40:
            print("ℹ️ ADVISORY: Moderate risk conditions")
            print("   • Monitor weather updates regularly")
            print("   • Be prepared for sudden weather changes")
            print("   • Avoid unnecessary outdoor activities\n")
        else:
            print("✅ Normal weather conditions")
            print("   • Continue regular activities")
            print("   • Standard precautions apply\n")

        print(f"{'='*60}\n")

        return {
            'location': full_address,
            'coordinates': (lat, lon),
            'cloudburst_risk': cloudburst_risk,
            'flood_risk': flood_risk,
            'weather_data': weather_data
        }

In [6]:
print("=" * 70)
print("   CLOUD BURST & FLASH FLOOD DETECTION SYSTEM")
print("   For Himachal Pradesh & Uttarakhand")
print("=" * 70)
print("\nInitializing system...")

# Create system instance
detector = CloudBurstDetectionSystem()

print("✅ System initialized successfully!\n")
print("=" * 70)

   CLOUD BURST & FLASH FLOOD DETECTION SYSTEM
   For Himachal Pradesh & Uttarakhand

Initializing system...
✅ System initialized successfully!



In [ ]:
def run_detection():
    """
    Run the detection system with user input
    """
    while True:
        place_name = input("\n🌍 Enter location name (or 'exit' to quit): ").strip()

        if place_name.lower() == 'exit':
            print("\n👋 Thank you for using the Cloud Burst Detection System!")
            break

        if not place_name:
            print("⚠️ Please enter a valid location name")
            continue

        # Analyze the location
        result = detector.analyze_location(place_name)

        # Ask if user wants to check another location
        continue_check = input("\n❓ Check another location? (yes/no): ").strip().lower()
        if continue_check not in ['yes', 'y']:
            print("\n👋 Thank you for using the Cloud Burst Detection System!")
            break

# Run the interactive system
run_detection()


🌍 Enter location name (or 'exit' to quit): Dehradun

🔍 Analyzing weather conditions for: Dehradun
⏳ Fetching real-time data...

✅ Location found: Dehradun, Uttarakhand, 248001, India
   Coordinates: 30.3256°N, 78.0437°E


📍 LOCATION: Dehradun, Uttarakhand, 248001, India
🕐 TIME: 2025-10-04T07:15

🌡️ CURRENT WEATHER CONDITIONS:
   Temperature: 22.2°C
   Humidity: 86%
   Precipitation: 0.0mm
   Rain: 0.0mm
   Cloud Cover: 64%
   Wind Speed: 4.4 km/h
   Wind Gusts: 9.4 km/h
   Pressure (MSL): 1010.5 hPa
   Surface Pressure: 934.8 hPa


⚠️ RISK ASSESSMENT:

☁️ CLOUD BURST RISK: 43.0%
   Status: 🟡 MODERATE - Monitor Conditions

🌊 FLASH FLOOD RISK: 34.61%
   Status: 🟢 LOW - Normal Conditions

📊 RISK INTERPRETATION:

ℹ️ ADVISORY: Moderate risk conditions
   • Monitor weather updates regularly
   • Be prepared for sudden weather changes
   • Avoid unnecessary outdoor activities


